In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import torch
from Models import *
from functions import *

In [2]:
x = np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', skip_header=1, usecols=range(1, 17), dtype=float)
y = np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', skip_header=1, usecols=(1,2),  dtype=float)

In [3]:
print(x.shape)
print(y.shape)
print(x[:,0].shape)
print(y[:,0].shape)

(6510, 16)
(6510, 2)
(6510,)
(6510,)


In [4]:
import xgboost as xgb

model = xgb.XGBRegressor()
 
model.fit(x, y)

importances = model.feature_importances_
print(importances)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

In [ ]:
input_size = x.shape[1]
hidden_size = 500
output_size = y.shape[1]
print(input_size)
print(output_size)

# model = MLP(input_size, hidden_size, output_size)
model = MLP_v2(input_size, hidden_size, output_size)

In [ ]:

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 400

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch) % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')